In [ ]:
# Ideas de modelos

# Weighted Scoring (Puntuación Ponderada):
# Modelo propio en el cual se calcule un puntaje del vino según diversas características y
# el algoritmo busque maximizar tal puntaje considerando el input del usuario.
# Entra en la categoría de Content-Based Filtering y Rule-Based Scoring.

# ¿De qué se compondría el puntaje?
# 1) rating (proxy de calidad)
# 2) zscore_quality_price (relación de calidad al precio)
# 3) rating_qty (popularidad del vino)
# 4) Similitud a preferencias de usuario



# Además, podría ser que si el usuario no pone perfil de sabor que le clusterizemos el vino y le demos alguno que
# Tenga un perfil de sabor parecido para ese pairing o grupo de pairings.

In [21]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)
wines = pd.read_csv("../src/data/transformed/wines_clean.csv")
wines.head(3)

,wine_link,name,year,winery,rating,rating_qty,price,body,tannis,sweetness,acidity,style,alcohol,image,ageing,black fruit,citrus,dried fruit,earthy,floral,oaky,red fruit,spices,tree fruit,tropical,vegetal,yeasty,any junk food will do,aperitif,appetizers and snacks,beef,blue cheese,cured meat,"game (deer, venison)",goat's milk cheese,lamb,lean fish,mature and hard cheese,mild and soft cheese,mushrooms,pasta,pork,poultry,"rich fish (salmon, tuna etc)",shellfish,spicy food,veal,vegetarian,Albariño,Barbera,Bonarda,Béquignol Noir,Cabernet Franc,Cabernet Sauvignon,Cereza,Chardonnay,Chenin Blanc,Criolla Grande,Garnacha,Gewürztraminer,Grenache,Grüner Veltliner,Malbec,Malvasia,Marsanne,Mencia,Merlot,Moscatel,Mourvedre,Pais,Pedro Ximenez,Petit Verdot,Pinot Gris,Pinot Noir,Riesling,Roussanne,Sangiovese,Sauvignon Blanc,Shiraz/Syrah,Sémillon,Tannat,Tempranillo,Torrontés,Trousseau,Verdejo,Viognier,Agrelo,Argentina,Brazil,Cafayate Valley,Calchaqui Valley,Campanha,Famatina,Gualtallary,La Consulta,La Rioja,Las Compuertas,Lujan de Cuyo,Lunlunta,Maipu,Mendoza,Paraje Altamira,Patagonia,Pedernal Valley,Perdriel,Rio Grande do Sul,Rio Negro,Salta,San Carlos,San Juan,San Rafael,Serra Gaúcha,Tulum Valley,Tunuyán,Tupungato,Uco Valley,Vale dos Vinhedos,Vista Flores
0,https://www.vivino.com/US/en/luigi-bosca-parai...,Paraiso,2020.0,Luigi Bosca,4.8,582.0,188.33,0.7343,0.5090,0.1361,0.4474,Argentinian Cabernet Sauvignon - Malbec,0.1410,https://images.vivino.com/thumbs/_Bf6JTwYRpSX6...,0.0000,0.3500,0.000,0.000,0.1250,0.0500,0.3250,0.0500,0.1000,0.0,0.0,0.000,0.000,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,https://www.vivino.com/US/en/catena-zapata-est...,Estiba Reservada,2015.0,Catena Zapata,4.7,297.0,675.00,0.7417,0.5583,0.1434,0.5445,Argentinian Bordeaux Blend,0.1400,https://images.vivino.com/thumbs/Yt464jw0QS-ug...,0.0241,0.2008,0.008,0.012,0.0964,0.0241,0.4378,0.0843,0.1004,0.0,0.0,0.004,0.008,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,https://www.vivino.com/US/en/catena-zapata-est...,Estiba Reservada,2017.0,Catena Zapata,4.7,219.0,580.00,0.7417,0.5583,0.1434,0.5445,Argentinian Bordeaux Blend,0.1416,https://images.vivino.com/thumbs/Yt464jw0QS-ug...,0.0241,0.2008,0.008,0.012,0.0964,0.0241,0.4378,0.0843,0.1004,0.0,0.0,0.004,0.008,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
from scipy import stats
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import MinMaxScaler

wines_copy = wines.copy()

user_input = {
    "pairing_list": ["beef", "pasta"],
    "precio_min": 20,
    "precio_max": 100,
    "tastes": {
        "body": 0.1,
        "tannins": 0.1,
        "sweetness": 0.5,
        "acidity": 0.1
    }
}

# Filtro por pairings
user_pairings = user_input["pairing_list"]
wine_base = wines_copy[wines_copy[user_pairings].sum(axis=1)>0]

# Filtro por precio
wine_base = wine_base[wine_base["price"]>=user_input["precio_min"]]
wine_base = wine_base[wine_base["price"]<=user_input["precio_max"]]

# Zscore quality/price
wine_base['rating_zscore'] = stats.zscore(wine_base["rating"])
wine_base['price_zscore'] = stats.zscore(wine_base["price"])
wine_base["zscore_quality_price"] = wine_base['rating_zscore'] - wine_base['price_zscore']

# User similarity (Euclidean Distances)
scaler = MinMaxScaler()

taste_features = ["body", "tannins", "sweetness", "acidity"]
scaled_tastes = [taste + "_scld" for taste in taste_features]

wine_base[scaled_tastes] = scaler.fit_transform(wine_base[taste_features])
user_tastes = np.array(list(user_input["tastes"].values())).reshape(1,-1)
distances = euclidean_distances(user_tastes, wine_base[scaled_tastes])[0]
wine_base["user_similarity"] = 1 - (distances / distances.max())

# Puntaje sintético del vino
wine_base["wine_score"] = (
    0.5 * (wine_base["rating"] / 5) + 
    0.3 * (wine_base["zscore_quality_price"] / wine_base["zscore_quality_price"].max()) + 
    0.2 * (wine_base["rating_qty"] / wine_base["rating_qty"].max()) +
    0.1 * wine_base["user_similarity"]
)

wine_base.nlargest(3,"wine_score")

,wine_link,name,year,winery,rating,rating_qty,price,body,tannis,sweetness,acidity,style,alcohol,image,ageing,black fruit,citrus,dried fruit,earthy,floral,oaky,red fruit,spices,tree fruit,tropical,vegetal,yeasty,any junk food will do,aperitif,appetizers and snacks,beef,blue cheese,cured meat,"game (deer, venison)",goat's milk cheese,lamb,lean fish,mature and hard cheese,mild and soft cheese,mushrooms,pasta,pork,poultry,"rich fish (salmon, tuna etc)",shellfish,spicy food,veal,vegetarian,Albariño,Barbera,Bonarda,Béquignol Noir,Cabernet Franc,Cabernet Sauvignon,Cereza,Chardonnay,Chenin Blanc,Criolla Grande,Garnacha,Gewürztraminer,Grenache,Grüner Veltliner,Malbec,Malvasia,Marsanne,Mencia,Merlot,Moscatel,Mourvedre,Pais,Pedro Ximenez,Petit Verdot,Pinot Gris,Pinot Noir,Riesling,Roussanne,Sangiovese,Sauvignon Blanc,Shiraz/Syrah,Sémillon,Tannat,Tempranillo,Torrontés,Trousseau,Verdejo,Viognier,Agrelo,Argentina,Brazil,Cafayate Valley,Calchaqui Valley,Campanha,Famatina,Gualtallary,La Consulta,La Rioja,Las Compuertas,Lujan de Cuyo,Lunlunta,Maipu,Mendoza,Paraje Altamira,Patagonia,Pedernal Valley,Perdriel,Rio Grande do Sul,Rio Negro,Salta,San Carlos,San Juan,San Rafael,Serra Gaúcha,Tulum Valley,Tunuyán,Tupungato,Uco Valley,Vale dos Vinhedos,Vista Flores,rating_zscore,price_zscore,zscore_quality_price,body_scld,tannis_scld,sweetness_scld,acidity_scld,user_similarity,wine_score
1159,https://www.vivino.com/el-enemigo-malbec/w/167...,Malbec,2016.0,El Enemigo,4.4,7869.0,25.00,0.6494,0.3528,0.2076,0.3493,Argentinian Mendoza Malbec Red,0.1402,https://images.vivino.com/thumbs/S0m-9WKZRfKqj...,0.0080,0.3148,0.005,0.0118,0.1340,0.0183,0.3422,0.0791,0.0664,0.0049,0.0004,0.0040,0.0111,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.573989,-0.657279,2.231269,0.631927,0.406818,0.512719,0.110657,0.598057,0.854768
1132,https://www.vivino.com/falasco-wines-bressano-...,Bressano Malbec,2018.0,Falasco Wines,4.5,769.0,29.24,0.7227,0.4589,0.1516,0.3820,Argentinian Uco Valley Malbec Red,0.1402,https://images.vivino.com/thumbs/x6Z_JjP_Qz-a9...,0.0000,0.2778,0.000,0.0000,0.0000,0.0000,0.3333,0.1667,0.2222,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2.057436,-0.440543,2.497978,0.766422,0.567576,0.374413,0.168739,0.459153,0.804417
1437,https://www.vivino.com/catena-malbec/w/1873?ye...,Malbec,2019.0,Catena,4.1,18091.0,20.99,0.6504,0.2626,0.1931,0.3378,Argentinian Mendoza Malbec Red,0.1350,https://images.vivino.com/thumbs/SPgFSZm0Tm-O-...,0.0068,0.3029,0.003,0.0111,0.1414,0.0213,0.3244,0.1059,0.0589,0.0034,0.0005,0.0057,0.0145,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.123650,-0.862259,0.985909,0.633761,0.270152,0.476908,0.090231,0.633069,0.791712


1. Crear datos sintéticos de gustó o no gustó al usuario.
2. Entrenar modelos en base a tales datos sintéticos (RandomForest, Redes Neuronales).
3. Incluir que el usuario ponga si le gustó o no el vino recomendado, para alimentar los algoritmos.